# Packages

In [1]:
import pandas as pd
import numpy as np
import os
import time
import folium

import keras

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as ticker
from matplotlib.ticker import PercentFormatter

pd.options.display.max_columns = None
pd.options.display.max_rows = None

Using TensorFlow backend.


In [2]:
from lib_Map import plot_map

# Load data

In [3]:
Data_train = pd.read_excel("./csv/Modelo de Consumo Predictivo.xlsx", sheet_name = "Data")
print(Data_train.shape)
Data_test = pd.read_excel("./csv/Modelo de Consumo Predictivo.xlsx", sheet_name = "Eval")
print(Data_test.shape)

(440806, 10)
(148931, 9)


In [4]:
Data_train.columns

Index(['ID', 'ARTICULO', 'FECHA', 'CENTROCONSUMO', 'UBICACIONVIRTUAL',
       'ID_ORGANOGESTOR', 'DESC_PROVINCIA', 'FECHACADUCIDADEXISTENCIAS',
       'IMPORTESALIDA', 'CANTIDADSALIDA'],
      dtype='object')

In [5]:
Data_test.columns

Index(['ID', 'ARTICULO', 'FECHA', 'CENTROCONSUMO', 'UBICACIONVIRTUAL',
       'ID_ORGANOGESTOR', 'DESC_PROVINCIA', 'FECHACADUCIDADEXISTENCIAS',
       'IMPORTESALIDA'],
      dtype='object')

### Separación año, mes y día:

In [6]:
Data_train['Año'] = [i.year for i in Data_train['FECHA']]
Data_train['Mes'] = [i.month for i in Data_train['FECHA']]
Data_train['Dia'] = [i.day for i in Data_train['FECHA']]

Data_test['Año'] = [i.year for i in Data_test['FECHA']]
Data_test['Mes'] = [i.month for i in Data_test['FECHA']]
Data_test['Dia'] = [i.day for i in Data_test['FECHA']]

## Data test y train mensuales:

### Train:

In [7]:
del Data_train['ID'], Data_train['FECHA'], Data_train['FECHACADUCIDADEXISTENCIAS'], Data_train['Dia'], Data_train['UBICACIONVIRTUAL']


In [9]:
ImporteSalidaSuma = (Data_train.groupby(('ARTICULO', 'CENTROCONSUMO', 'ID_ORGANOGESTOR', 'DESC_PROVINCIA', 'Año', 'Mes')) # Agrupar
          .IMPORTESALIDA    # Quedarse con la columna packet_length
          .apply(sum)       # Calcular su suma
          .reset_index())   # Deshacer índice jerárquico

CantidadSalidaSuma = (Data_train.groupby(('ARTICULO', 'CENTROCONSUMO', 'ID_ORGANOGESTOR', 'DESC_PROVINCIA', 'Año', 'Mes')) # Agrupar
          .CANTIDADSALIDA    # Quedarse con la columna packet_length
          .apply(sum)       # Calcular su suma
          .reset_index())   # Deshacer índice jerárquico

C:\Users\victo\Anaconda3\envs\baseFSA\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.
C:\Users\victo\Anaconda3\envs\baseFSA\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  


In [13]:
lista_cols = ['ARTICULO', 'CENTROCONSUMO', 'ID_ORGANOGESTOR', 'DESC_PROVINCIA', 'Año', 'Mes']
Data_train_mensual = pd.merge(ImporteSalidaSuma, CantidadSalidaSuma,  how='inner', on= lista_cols)
Data_train_mensual.shape
# Ordenar = Data_train_mensual.sort_values(['Año', 'Mes', 'ARTICULO', 'CENTROCONSUMO' ])

(67688, 8)

In [14]:
Data_train_mensual.head()

,ARTICULO,CENTROCONSUMO,ID_ORGANOGESTOR,DESC_PROVINCIA,Año,Mes,IMPORTESALIDA,CANTIDADSALIDA
0,-1073449806,1021,4920000000000360471,Granada,2012,3,1.324,2.0
1,-1073449806,9517,4920000000000360471,Granada,2012,1,0.790,2.0
2,-1073449806,9517,4920000000000360471,Granada,2012,4,1.324,2.0
3,-1073449806,9517,4920000000000360471,Granada,2012,5,0.662,1.0
4,-1073449806,9538,4920000000000360471,Granada,2012,1,0.000,-20.0


### Test

In [15]:
del Data_test['ID'], Data_test['FECHA'], Data_test['FECHACADUCIDADEXISTENCIAS'], Data_test['Dia'],  Data_test['UBICACIONVIRTUAL']

In [16]:
Data_test_mensual = (Data_test.groupby(('ARTICULO', 'CENTROCONSUMO', 'ID_ORGANOGESTOR', 'DESC_PROVINCIA', 'Año', 'Mes')) # Agrupar
          .IMPORTESALIDA    # Quedarse con la columna packet_length
          .apply(sum)       # Calcular su suma
          .reset_index())   # Deshacer índice jerárquico
Data_test_mensual.shape

C:\Users\victo\Anaconda3\envs\baseFSA\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


(22795, 7)

In [17]:
Data_test_mensual.head()

,ARTICULO,CENTROCONSUMO,ID_ORGANOGESTOR,DESC_PROVINCIA,Año,Mes,IMPORTESALIDA
0,-1073438649,73921,7920000000000360461,Málaga,2016,2,76.324500
1,-1073438649,73921,7920000000000360461,Málaga,2016,4,32.945844
2,-1073438649,73921,7920000000000360461,Málaga,2016,6,21.802000
3,-1073438649,73921,7920000000000360461,Málaga,2016,7,10.900500
4,-1073438649,73921,7920000000000360461,Málaga,2016,8,21.800000


### Limpieza

In [25]:
len(Data_train_mensual['ARTICULO'].unique().tolist())

20

In [26]:
len(Data_test_mensual['ARTICULO'].unique().tolist())

11

In [27]:
l=[]
for i in Data_test_mensual['ARTICULO'].unique().tolist():
    if i not in Data_train_mensual['ARTICULO'].unique().tolist():
        l.append(i)
len(l)

1

# Map

In [9]:
coord_path = "./csv/listado-longitud-latitud-municipios-espana.xls"
TypeData = Data_train

### Total:

In [6]:
plot_map(coord_path, TypeData, Año = 'Total')

### Por años:

In [8]:
for Año in Data_train['Año'].unique().tolist():
    plot_map(coord_path, TypeData, Año)